# Building Machine Learning Classifiers: Evaluate Random Forest with GridSearchCV

**Grid-search:** Exhaustively search all parameter combinations in a given grid to determine the best model.

**Cross-validation:** Divide a dataset into k subsets and repeat the holdout method k times where a different subset is used as the holdout set in each iteration.

### Read in text

In [2]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import string
nltk.download('stopwords')

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

# --------- Here we are using 2 vectorizing frameworks to see which of these framework works better

# TF-IDF
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])
X_tfidf_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)

# CountVectorizer
count_vect = CountVectorizer(analyzer=clean_text)
X_count = count_vect.fit_transform(data['body_text'])
X_count_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_count.toarray())], axis=1)

X_count_feat.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,body_len,punct%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,49,4.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,62,3.2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,28,7.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,135,4.4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
# Convert column names to strings
X_tfidf_feat.columns = X_tfidf_feat.columns.astype(str)
X_count_feat.columns = X_count_feat.columns.astype(str)

### Exploring parameter settings using GridSearchCV

In [4]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [5]:
rf = RandomForestClassifier()
param = {'n_estimators': [10, 150, 300],
        'max_depth': [30, 60, 90, None]}

gs = GridSearchCV(rf, param, cv=5, n_jobs=-1)
gs_fit = gs.fit(X_tfidf_feat, data['label'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
10,21.350854,2.145029,0.293777,0.075812,None,150,"{'max_depth': None, 'n_estimators': 150}",0.979354,0.980251,0.973944,0.967655,0.973046,0.974850,0.004589,1
8,35.687660,1.158413,0.411395,0.133364,90,300,"{'max_depth': 90, 'n_estimators': 300}",0.977558,0.974865,0.974843,0.969452,0.972147,0.973773,0.002756,2
11,38.307245,5.959903,0.382131,0.085821,None,300,"{'max_depth': None, 'n_estimators': 300}",0.977558,0.974865,0.974843,0.968553,0.973046,0.973773,0.002981,2
5,32.509074,3.252289,0.312449,0.010131,60,300,"{'max_depth': 60, 'n_estimators': 300}",0.975763,0.974865,0.974843,0.967655,0.971249,0.972875,0.003035,4
7,18.913120,0.923332,0.314911,0.104206,90,150,"{'max_depth': 90, 'n_estimators': 150}",0.978456,0.977558,0.971249,0.968553,0.968553,0.972874,0.004315,5


In [6]:
rf = RandomForestClassifier()
param = {'n_estimators': [10, 150, 300],
        'max_depth': [30, 60, 90, None]}

gs = GridSearchCV(rf, param, cv=5, n_jobs=-1)
gs_fit = gs.fit(X_count_feat, data['label'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
11,36.171515,4.802755,0.473929,0.193697,None,300,"{'max_depth': None, 'n_estimators': 300}",0.976661,0.975763,0.974843,0.966757,0.970350,0.972875,0.003751,1
7,17.939712,0.178437,0.258091,0.012825,90,150,"{'max_depth': 90, 'n_estimators': 150}",0.976661,0.973968,0.973046,0.968553,0.971249,0.972695,0.002710,2
10,19.660399,0.718288,0.289435,0.068703,None,150,"{'max_depth': None, 'n_estimators': 150}",0.976661,0.973070,0.972147,0.969452,0.970350,0.972336,0.002511,3
8,37.183125,0.620040,0.415699,0.125588,90,300,"{'max_depth': 90, 'n_estimators': 300}",0.976661,0.972172,0.973944,0.967655,0.969452,0.971977,0.003191,4
4,14.769456,0.148934,0.224687,0.003332,60,150,"{'max_depth': 60, 'n_estimators': 150}",0.977558,0.973968,0.970350,0.964960,0.969452,0.971258,0.004261,5
